In [14]:
import os, sys
import re
import json
import glob
import psycopg2
from psycopg2 import sql
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
from sqlalchemy import create_engine

In [15]:
rpath = os.path.abspath('..')
if rpath not in sys.path:
    sys.path.insert(0, rpath)

from src.loader import SlackDataLoader
import src.utils as utils

In [17]:
db_params = {
    'host': '127.0.0.1',
    'user': 'postgres',
    'password': 'postgres',
    'port': '5432'
}
connection = psycopg2.connect(**db_params)
connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
cursor = connection.cursor()
db_name = '10_ac_week_0'
quoted_db_name = sql.Identifier(db_name)
create_db_query = sql.SQL("CREATE DATABASE {}").format(quoted_db_name)
cursor.execute(create_db_query)

In [5]:
schema_file_path = 'schema.sql'

In [7]:
with open(schema_file_path, 'r') as schema_file:
    schema_sql = schema_file.read()

In [18]:
# Construct the connection string
connection_string = f"postgresql+psycopg2://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_name}"

# Create the SQLAlchemy engine
engine = create_engine(connection_string)

In [20]:
# Please include the path to datafiles here for this code to work
read_file = SlackDataLoader("../data")
message_with_user = read_file.slack_parser("../data/all-week8/")
selected_columns = ['msg_type', 'sender_name', 'msg_dist_type', 'reply_count']
df_selected = message_with_user[selected_columns]
table_name = "messages"
df_selected.to_sql(table_name, engine, index=False, if_exists='replace')
print(f"\nData saved to table '{table_name}' in the database.")


Data saved to table 'messages' in the database.
